# Homework 2

# Set up

## Installing packages

In [ ]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [ ]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
# DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [ ]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 13.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 10.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 7.66MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 61.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 47.8MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [ ]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [ ]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [ ]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------


from langchain_openai import ChatOpenAI
DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
llm = ChatOpenAI(
  model="deepseek-v3.2-251201",          # or "deepseek-reasoner"
  api_key=DEEPSEEK_API_KEY,
  base_url="https://api.zetatechs.com/v1",
  temperature=0,
 )

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content="I'll start by saying hello to Bao, then search for Alice on Facebook.\n\n" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 3181, 'total_tokens': 3241, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'deepseek-v3.2', 'system_fingerprint': None, 'id': 'chatcmpl-d4fa3a56-f200-987f-bbc2-e285aa535dff', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019ca28c-d34b-7480-9290-bb094a9dd8ae-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'call_dc53064f1f6d4e4d8c30f3c2', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 3181, 'output_tokens': 60, 'total_tokens': 3241, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


In [ ]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
# await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
# await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
# await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
# await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
# await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
# await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
await tools[5].ainvoke({'person_id': 456})

[{'type': 'text',
  'text': '{"profile_id":456,"post_count":4,"total_likes":5,"liked_by":[4390,3622,7500,4269,8464],"engagement_score":1.25}',
  'id': 'lc_fa190d86-ff4c-4402-a76c-10073542ad1e'}]

# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [ ]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [ ]:
import os
import json
import re
import difflib
import asyncio
import logging

_mcp_logger = logging.getLogger('mcp.client.streamable_http')
_mcp_logger.setLevel(logging.CRITICAL)
_mcp_logger.propagate = False
logging.getLogger('mcp').setLevel(logging.CRITICAL)

from langchain_core.messages import HumanMessage

def _clamp01(x, default=0.5):
    try:
        x = float(x)
    except Exception:
        return float(default)
    return max(0.0, min(1.0, x))

def _strip_code_fences(text):
    text = (text or '').strip()
    if not text.startswith('```'):
        return text
    lines = text.splitlines()
    if lines and lines[0].startswith('```'):
        lines = lines[1:]
    if lines and lines[-1].startswith('```'):
        lines = lines[:-1]
    return chr(10).join(lines).strip()

def _extract_json(text):
    text = _strip_code_fences(text)
    start = text.find('{')
    end = text.rfind('}')
    if start == -1 or end == -1 or end <= start:
        raise ValueError('No JSON object found in LLM output')
    return json.loads(text[start:end + 1])

def _pick_tool(tool_list, *hints):
    for hint in hints:
        for t in tool_list:
            if getattr(t, 'name', None) == hint:
                return t
    for hint in hints:
        for t in tool_list:
            if hint in getattr(t, 'name', ''):
                return t
    names = [getattr(t, 'name', None) for t in tool_list]
    raise KeyError(f'Tool not found. Hints={hints}. Available={names}')

_MCP_MAX_CONCURRENCY = int(os.getenv('MCP_MAX_CONCURRENCY', '1') or '1')
_MCP_SEM = asyncio.Semaphore(max(1, _MCP_MAX_CONCURRENCY))

def _unwrap_mcp_output(raw):
    if raw is None:
        return None

    if isinstance(raw, list) and raw and isinstance(raw[0], dict) and 'text' in raw[0]:
        txt = raw[0].get('text')
        if isinstance(txt, str):
            s = txt.strip()
            if s and s[0] in '{[':
                try:
                    return json.loads(s)
                except Exception:
                    return raw

    if isinstance(raw, dict) and 'text' in raw and isinstance(raw.get('text'), str):
        s = raw['text'].strip()
        if s and s[0] in '{[':
            try:
                return json.loads(s)
            except Exception:
                return raw

    if isinstance(raw, str):
        s = raw.strip()
        if s and s[0] in '{[':
            try:
                return json.loads(s)
            except Exception:
                return raw

    return raw

def _is_transient_mcp_error(e: Exception) -> bool:

    sub = getattr(e, 'exceptions', None)
    if isinstance(sub, (list, tuple)) and sub:
        for se in sub:
            if isinstance(se, Exception) and _is_transient_mcp_error(se):
                return True

    name = type(e).__name__.lower()
    msg = str(e).lower()
    if 'server disconnected' in msg:
        return True
    if 'all connection attempts failed' in msg:
        return True
    if 'connection reset' in msg or 'reset by peer' in msg:
        return True
    if 'timed out' in msg or 'timeout' in msg:
        return True
    if 'remotep' in name or 'protocol' in name:
        return True
    if 'connect' in name and 'error' in name:
        return True
    return False

async def _ainvoke_with_retry(tool, payload, default=None, retries=4, timeout_s=60, raise_on_fail=False):
    last_exc = None
    tool_name = getattr(tool, 'name', None)
    for attempt in range(int(retries)):
        try:
            async with _MCP_SEM:
                raw = await asyncio.wait_for(tool.ainvoke(payload), timeout=timeout_s)
            return _unwrap_mcp_output(raw)
        except Exception as e:
            last_exc = e
            if _is_transient_mcp_error(e) and tool_name:
                try:
                    refreshed = await _ensure_tools(force_refresh=True)
                    if refreshed:
                        tool = _pick_tool(refreshed, tool_name)
                except Exception:
                    pass
            if attempt < int(retries) - 1:
                await asyncio.sleep(min(6.0, 0.75 * (2 ** attempt)))
    if raise_on_fail and last_exc is not None:
        raise last_exc
    return default

_md = None
def _read_pdf_text(pdf_path):
    global _md
    try:
        from markitdown import MarkItDown
        if _md is None:
            _md = MarkItDown(enable_plugins=False)
        result = _md.convert(pdf_path)
        return (getattr(result, 'text_content', None) or '').strip()
    except Exception:
        try:
            import PyPDF2
            reader = PyPDF2.PdfReader(pdf_path)
            parts = []
            for p in reader.pages:
                parts.append(p.extract_text() or '')
            return chr(10).join(parts).strip()
        except Exception:
            return ''

def _cv_snippet(text, max_chars=12000):
    text = text or ''
    if len(text) <= max_chars:
        return text
    head = text[:8000]
    tail = text[-4000:]
    return head + chr(10) + '...' + chr(10) + tail

def _find_cv_paths():
    paths = []
    for i in range(1, 6):
        direct = [f'CV_{i}.pdf', os.path.join('downloaded_cvs', f'CV_{i}.pdf')]
        found = None
        for p in direct:
            if os.path.exists(p):
                found = p
                break
        if found is None:
            for folder in ['.', 'downloaded_cvs']:
                if not os.path.isdir(folder):
                    continue
                for fn in os.listdir(folder):
                    if not fn.lower().endswith('.pdf'):
                        continue
                    digits = ''.join([ch for ch in fn if ch.isdigit()])
                    if digits and digits.isdigit() and int(digits) == i:
                        found = os.path.join(folder, fn)
                        break
                if found:
                    break
        if found is None:
            raise FileNotFoundError(f'Missing CV_{i}.pdf (or equivalent) for i={i}')
        paths.append(found)
    return paths

def _similarity(a, b):
    a = (a or '').lower().strip()
    b = (b or '').lower().strip()
    if not a or not b:
        return 0.0
    return difflib.SequenceMatcher(None, a, b).ratio()

def _strip_parenthetical(text):
    if not isinstance(text, str):
        return ''
    out = []
    depth = 0
    for ch in text:
        if ch == '(':
            depth += 1
            continue
        if ch == ')' and depth:
            depth -= 1
            continue
        if depth == 0:
            out.append(ch)
    return ''.join(out)

def _tokenize_location(text):
    text = _strip_parenthetical(text or '')
    for sep in ['|', '/', ';', chr(10)]:
        text = text.replace(sep, ',')
    parts = [p.strip() for p in text.split(',')]

    tokens = []
    seen = set()
    for p in parts:
        if not p:
            continue
        p = ' '.join(p.split())
        key = p.lower()
        if key in seen:
            continue
        seen.add(key)
        tokens.append(p)
    return tokens

def _location_candidates(candidate):
    if not isinstance(candidate, dict):
        return []
    tokens = []
    for k in ('city', 'country', 'location_query'):
        tokens.extend(_tokenize_location(candidate.get(k)))
    return tokens[:4]

def _as_list(x):
    if x is None:
        return []
    if isinstance(x, list):
        return x
    return [x]

def _pick_str(x, *keys):
    if isinstance(x, str) and x.strip():
        return x.strip()
    if isinstance(x, dict):
        for k in keys:
            v = x.get(k)
            if isinstance(v, str) and v.strip():
                return v.strip()
    return None

def _dedup_strings(items, limit=None):
    out = []
    seen = set()
    for s in items:
        if not isinstance(s, str):
            continue
        s = s.strip()
        if not s:
            continue
        key = s.lower()
        if key in seen:
            continue
        seen.add(key)
        out.append(s)
        if limit and len(out) >= int(limit):
            break
    return out

_YEAR_RE = re.compile(r'\b(19|20)\d{2}\b')
_YEAR_RANGE_RE = re.compile(r'\b(19|20)\d{2}\s*[-–—]\s*(?:present|current|now|\d{2,4})\b', re.IGNORECASE)

def _strip_year_tokens(text):
    if not isinstance(text, str):
        return ''
    t = text
    t = _YEAR_RANGE_RE.sub('', t)
    t = _YEAR_RE.sub('', t)
    t = re.sub(r'\b(graduated|present|current|now)\b', '', t, flags=re.IGNORECASE)
    t = t.replace('–', ' ').replace('—', ' ').replace('-', ' ')
    return ' '.join(t.split()).strip()

def _extract_company_from_experience_line(line):
    if not isinstance(line, str):
        return None
    s = ' '.join(line.replace(chr(10), ' ').split()).strip()
    if not s:
        return None

    company = None
    if ',' in s:
        parts = [p.strip() for p in s.split(',') if p.strip()]
        if len(parts) >= 2:
            company = parts[1]
    else:
        m = re.search(r'\b(?:at|@)\s+([A-Z][\w& .-]{1,80})\b', s)
        if m:
            company = m.group(1)

    company = _strip_year_tokens(company) if company else ''
    company = company.strip(' .|/')
    if not company:
        return None

    if len(company) > 80:
        return None

    if _YEAR_RE.search(company):
        company = _YEAR_RE.sub('', company).strip()

    return company or None

def _extract_school_from_education_line(line):
    if not isinstance(line, str):
        return None
    s = ' '.join(line.replace(chr(10), ' ').split()).strip()
    if not s:
        return None

    school = None
    if ',' in s:
        parts = [p.strip() for p in s.split(',') if p.strip()]
        for p in parts:
            if re.search(r'(university|college|institute|school|polyu|nus|hku)', p, flags=re.IGNORECASE):
                school = p
                break
        if school is None and len(parts) >= 2:
            school = parts[1]
    else:
        m = re.search(r'([A-Z][A-Za-z& .-]{2,80}(?:University|College|Institute|School))\b', s)
        if m:
            school = m.group(1)

    school = _strip_year_tokens(school) if school else ''
    school = school.strip(' .|/')
    if not school or len(school) > 100:
        return None
    return school

def _candidate_companies(candidate):
    if not isinstance(candidate, dict):
        return []
    companies = []
    cc = _pick_str(candidate.get('current_company'))
    if cc:
        companies.append(cc)
    for item in _as_list(candidate.get('experience')):
        if isinstance(item, dict):
            c = _pick_str(item, 'company', 'organization', 'employer')
            if c:
                companies.append(c)
        elif isinstance(item, str):
            c = _extract_company_from_experience_line(item)
            if c:
                companies.append(c)
    return _dedup_strings(companies, limit=8)

def _candidate_schools(candidate):
    if not isinstance(candidate, dict):
        return []
    schools = []
    for item in _as_list(candidate.get('education')):
        if isinstance(item, dict):
            s = _pick_str(item, 'school', 'university', 'institution', 'name')
            if s:
                schools.append(s)
        elif isinstance(item, str):
            s = _extract_school_from_education_line(item)
            if s:
                schools.append(s)
    return _dedup_strings(schools, limit=8)

def _candidate_skills(candidate):
    if not isinstance(candidate, dict):
        return []
    skills = []
    for item in _as_list(candidate.get('skills')):
        s = _pick_str(item, 'name', 'skill')
        if s:
            skills.append(s)
    return _dedup_strings(skills, limit=12)

def _linkedin_companies(profile):
    if not isinstance(profile, dict):
        return []
    companies = []
    for e in _as_list(profile.get('experience')):
        c = _pick_str(e, 'company')
        if c:
            companies.append(c)
    return _dedup_strings(companies, limit=12)

def _linkedin_schools(profile):
    if not isinstance(profile, dict):
        return []
    schools = []
    for e in _as_list(profile.get('education')):
        s = _pick_str(e, 'school')
        if s:
            schools.append(s)
    return _dedup_strings(schools, limit=12)

def _linkedin_skills(profile):
    if not isinstance(profile, dict):
        return []
    skills = []
    for item in _as_list(profile.get('skills')):
        s = _pick_str(item, 'name')
        if s:
            skills.append(s)
    return _dedup_strings(skills, limit=20)

def _max_similarity(list_a, list_b):
    best = 0.0
    for a in list_a:
        for b in list_b:
            best = max(best, _similarity(a, b))
    return float(best)

def _consistency_linkedin(candidate, profile):
    if not isinstance(candidate, dict) or not isinstance(profile, dict):
        return 0.5

    score = 0.5

    cand_companies = _candidate_companies(candidate)
    prof_companies = _linkedin_companies(profile)
    if cand_companies and prof_companies:
        m = _max_similarity(cand_companies, prof_companies)
        score += 0.25 if m >= 0.86 else -0.25

    cand_schools = _candidate_schools(candidate)
    prof_schools = _linkedin_schools(profile)
    if cand_schools and prof_schools:
        m = _max_similarity(cand_schools, prof_schools)
        score += 0.20 if m >= 0.86 else -0.20

    ct = _pick_str(candidate.get('current_title'))
    headline = _pick_str(profile.get('headline'))
    if ct and headline:
        m = _similarity(ct, headline)
        if m >= 0.75:
            score += 0.10
        elif m <= 0.35:
            score -= 0.10

    cand_city = _pick_str(candidate.get('city'))
    cand_country = _pick_str(candidate.get('country'))
    prof_city = _pick_str(profile.get('city'))
    prof_country = _pick_str(profile.get('country'))
    if cand_city and prof_city and _similarity(cand_city, prof_city) >= 0.85:
        score += 0.05
    if cand_country and prof_country and _similarity(cand_country, prof_country) >= 0.85:
        score += 0.03

    cand_skills = _candidate_skills(candidate)
    prof_skills = _linkedin_skills(profile)
    if cand_skills and prof_skills:
        m = _max_similarity(cand_skills, prof_skills)
        if m >= 0.86:
            score += 0.05

    return _clamp01(score, default=0.5)

def _consistency_facebook(candidate, profile):
    if not isinstance(candidate, dict) or not isinstance(profile, dict):
        return 0.5

    score = 0.5

    cand_city = _pick_str(candidate.get('city'))
    cand_country = _pick_str(candidate.get('country'))
    fb_city = _pick_str(profile.get('city'))
    fb_country = _pick_str(profile.get('country'))
    if cand_city and fb_city and _similarity(cand_city, fb_city) >= 0.85:
        score += 0.10
    if cand_country and fb_country and _similarity(cand_country, fb_country) >= 0.85:
        score += 0.05

    cand_company = _pick_str(candidate.get('current_company'))
    fb_company = _pick_str(profile.get('current_company'))
    if cand_company and fb_company:
        m = _similarity(cand_company, fb_company)
        score += 0.25 if m >= 0.86 else -0.25

    ct = _pick_str(candidate.get('current_title'))
    fb_job = _pick_str(profile.get('current_job'))
    if ct and fb_job:
        m = _similarity(ct, fb_job)
        if m >= 0.70:
            score += 0.10
        elif m <= 0.35:
            score -= 0.05

    return _clamp01(score, default=0.5)

def _compact_linkedin(profile):
    if not isinstance(profile, dict):
        return profile
    keys = [
        'id', 'name', 'headline', 'city', 'country', 'industry', 'status',
        'years_experience', 'summary', 'skills', 'experience', 'education'
    ]
    return {k: profile.get(k) for k in keys}

def _compact_facebook(profile):
    if not isinstance(profile, dict):
        return profile
    keys = [
        'id', 'display_name', 'original_name', 'city', 'country', 'hometown',
        'bio', 'status', 'education', 'current_job', 'current_company',
        'interests', 'friend_count'
    ]
    return {k: profile.get(k) for k in keys}

def _compact_interactions(interactions):
    if not isinstance(interactions, dict):
        return interactions
    keys = ['profile_id', 'post_count', 'total_likes', 'engagement_score']
    return {k: interactions.get(k) for k in keys}

def _ensure_llm():
    if globals().get('llm') is not None:
        return globals()['llm']

    api_key = globals().get('GEMINI_VERTEX_API_KEY') or os.getenv('VERTEX_API_KEY') or os.getenv('GEMINI_API_KEY')
    if api_key:
        from langchain_google_genai import ChatGoogleGenerativeAI
        return ChatGoogleGenerativeAI(model='gemini-2.0-flash', google_api_key=api_key, temperature=0)

    api_key = globals().get('DEEPSEEK_API_KEY') or os.getenv('DEEPSEEK_API_KEY')
    if api_key:
        from langchain_openai import ChatOpenAI
        return ChatOpenAI(
            model='deepseek-chat',
            api_key=api_key,
            base_url='https://api.deepseek.com/v1',
            temperature=0,
        )

    raise RuntimeError('No LLM API key found. Set VERTEX_API_KEY or DEEPSEEK_API_KEY.')

async def _ensure_tools(force_refresh=False):
    if not force_refresh and isinstance(globals().get('tools'), list) and globals()['tools']:
        return globals()['tools']
    from langchain_mcp_adapters.client import MultiServerMCPClient
    mcp_url = os.getenv('SOCIAL_GRAPH_MCP_URL', 'https://ftec5660.ngrok.app/mcp')
    client = MultiServerMCPClient({
        'social_graph': {
            'transport': 'http',
            'url': mcp_url,
            'headers': {'ngrok-skip-browser-warning': 'true'},
        }
    })
    last_exc = None
    for attempt in range(3):
        try:
            loaded = await asyncio.wait_for(client.get_tools(), timeout=30)
            globals()['tools'] = loaded
            return loaded
        except Exception as e:
            last_exc = e
            if attempt < 2:
                await asyncio.sleep(min(6.0, 0.75 * (2 ** attempt)))
    print(f"WARNING: cannot connect to MCP server at {mcp_url}: {last_exc}")
    return []

async def _extract_candidate(llm, cv_text):
    prompt = f'''
You are a KYC/recruiting verifier. Extract structured candidate information from the CV text.

Return ONLY a valid JSON object with keys:
full_name, name_variants, city, country, location_query, current_title, current_company,
education, experience, skills, keywords.

Rules:
- Use null if unknown
- Years must be integers or null
- Keep lists short (<= 6 items)
- name_variants should include likely variants of the name (<= 4)

CV_TEXT:
```
{_cv_snippet(cv_text)}
```
'''.strip()

    resp = llm.invoke([HumanMessage(content=prompt)])
    content = getattr(resp, 'content', '') if resp is not None else ''
    try:
        data = _extract_json(content)
    except Exception:
        data = {
            'full_name': None,
            'name_variants': [],
            'city': None,
            'country': None,
            'location_query': None,
            'current_title': None,
            'current_company': None,
            'education': [],
            'experience': [],
            'skills': [],
            'keywords': [],
        }

    if not isinstance(data.get('name_variants'), list):
        data['name_variants'] = []
    if data.get('full_name') and data['full_name'] not in data['name_variants']:
        data['name_variants'] = [data['full_name']] + data['name_variants']
    data['name_variants'] = [x for x in data['name_variants'] if isinstance(x, str) and x.strip()][:4]

    if not data.get('full_name'):
        for line in (cv_text or '').splitlines():
            line = line.strip()
            if not line:
                continue
            line_clean = line.replace('Name:', '').replace('姓名：', '').strip()
            if 2 <= len(line_clean) <= 80:
                data['full_name'] = line_clean
                if line_clean not in data['name_variants']:
                    data['name_variants'] = [line_clean] + data['name_variants']
                break

    return data

async def _choose_id_with_llm(llm, candidate, platform, options):
    prompt = f'''
You are matching a CV to a {platform} search result.

Candidate (from CV extraction):
{json.dumps(candidate, ensure_ascii=False)}

{platform} search options:
{json.dumps(options, ensure_ascii=False)}

Return ONLY JSON with keys: id (int or null), reason (string).
Choose null if none match reasonably.
'''.strip()

    resp = llm.invoke([HumanMessage(content=prompt)])
    content = getattr(resp, 'content', '') if resp is not None else ''
    try:
        out = _extract_json(content)
        return out.get('id', None)
    except Exception:
        return None

async def _best_linkedin(tools, llm, candidate):
    try:
        search = _pick_tool(tools, 'search_linkedin_people', 'linkedin_people', 'search_linkedin')
        get_profile = _pick_tool(tools, 'get_linkedin_profile', 'linkedin_profile', 'get_linkedin')
        get_interactions = _pick_tool(tools, 'get_linkedin_interactions', 'linkedin_interactions', 'get_interactions')
    except Exception:
        return None

    queries = []
    if candidate.get('name_variants'):
        queries.extend(candidate['name_variants'])
    if candidate.get('full_name'):
        queries.insert(0, candidate['full_name'])
    if candidate.get('keywords'):
        queries.extend(candidate['keywords'][:2])

    seen = set()
    queries = [q for q in queries if isinstance(q, str) and q.strip() and not (q in seen or seen.add(q))]
    queries = queries[:4]

    location_candidates = _location_candidates(candidate)

    results = []
    for q in queries:
        r = []
        for loc in location_candidates:
            payload = {'q': q, 'location': loc, 'limit': 20, 'fuzzy': True}
            try:
                r = await _ainvoke_with_retry(search, payload, default=[], retries=3, timeout_s=60, raise_on_fail=True)
            except Exception:
                r = await _ainvoke_with_retry(search, payload, default=[], retries=2, timeout_s=60)
            if isinstance(r, list) and r:
                break
        if not r:
            payload = {'q': q, 'limit': 20, 'fuzzy': True}
            try:
                r = await _ainvoke_with_retry(search, payload, default=[], retries=3, timeout_s=60, raise_on_fail=True)
            except Exception:
                r = await _ainvoke_with_retry(search, payload, default=[], retries=2, timeout_s=60)
        if isinstance(r, list):
            results.extend(r)

    seen_ids = set()
    dedup = []
    for r in results:
        if not isinstance(r, dict):
            continue
        rid = r.get('id')
        if rid is None or rid in seen_ids:
            continue
        seen_ids.add(rid)
        dedup.append(r)
    if not dedup:
        return None

    target_name = candidate.get('full_name') or (candidate.get('name_variants') or [None])[0]
    target_city = (candidate.get('city') or '').lower()
    target_country = (candidate.get('country') or '').lower()
    target_title = candidate.get('current_title') or ''

    scored = []
    for r in dedup:
        name = r.get('name') or ''
        loc = (r.get('location') or '').lower()
        s = 0.75 * _similarity(target_name, name)
        if target_city and target_city in loc:
            s += 0.10
        if target_country and target_country in loc:
            s += 0.15
        scored.append((s, r))
    scored.sort(key=lambda x: x[0], reverse=True)

    top5 = [r for _, r in scored[:5]]
    chosen_id = top5[0].get('id')

    if len(top5) >= 2:
        s0, s1 = scored[0][0], scored[1][0]
        if s0 < 0.85 and abs(s0 - s1) < 0.06:
            llm_id = await _choose_id_with_llm(llm, candidate, 'LinkedIn', top5)
            if isinstance(llm_id, int):
                chosen_id = llm_id

    if chosen_id is None:
        return None


    top_for_profiles = [r for _, r in scored[:12]]
    candidate_ids = [r.get('id') for r in top_for_profiles if r.get('id') is not None][:8]
    if not candidate_ids:
        candidate_ids = [chosen_id]
    prof_tasks = [
        _ainvoke_with_retry(get_profile, {'person_id': int(pid)}, default=None, retries=2, timeout_s=60)
        for pid in candidate_ids
    ]
    profiles = await asyncio.gather(*prof_tasks, return_exceptions=True)

    best_id = None
    best_profile = None
    best_match = -1.0
    for pid, prof in zip(candidate_ids, profiles):
        if isinstance(prof, Exception) or not isinstance(prof, dict):
            continue
        ms = _consistency_linkedin(candidate, prof)
        if ms > best_match:
            best_id = pid
            best_profile = prof
            best_match = ms

    if best_id is None:
        best_id = chosen_id
        best_profile = await _ainvoke_with_retry(get_profile, {'person_id': int(best_id)}, default=None, retries=2, timeout_s=60)
        best_match = _consistency_linkedin(candidate, best_profile) if isinstance(best_profile, dict) else 0.5

    if not isinstance(best_profile, dict) or best_match < 0.55:
        return None

    interactions = await _ainvoke_with_retry(get_interactions, {'person_id': int(best_id)}, default=None, retries=2, timeout_s=60)

    return {
        'person_id': int(best_id),
        'profile': best_profile,
        'interactions': interactions,
        'match_score': float(best_match),
    }

async def _best_facebook(tools, llm, candidate):
    try:
        search = _pick_tool(tools, 'search_facebook_users', 'facebook_users', 'search_facebook')
        get_profile = _pick_tool(tools, 'get_facebook_profile', 'facebook_profile', 'get_facebook')
    except Exception:
        return None

    queries = []
    if candidate.get('name_variants'):
        queries.extend(candidate['name_variants'])
    if candidate.get('full_name'):
        queries.insert(0, candidate['full_name'])

    seen = set()
    queries = [q for q in queries if isinstance(q, str) and q.strip() and not (q in seen or seen.add(q))]
    queries = queries[:3]

    results = []
    for q in queries:
        try:
            r = await _ainvoke_with_retry(search, {'q': q, 'limit': 20, 'fuzzy': True}, default=[], retries=3, timeout_s=60, raise_on_fail=True)
        except Exception:
            r = await _ainvoke_with_retry(search, {'q': q, 'limit': 20}, default=[], retries=2, timeout_s=60)
        if isinstance(r, list):
            results.extend(r)

    seen_ids = set()
    dedup = []
    for r in results:
        if not isinstance(r, dict):
            continue
        rid = r.get('id')
        if rid is None or rid in seen_ids:
            continue
        seen_ids.add(rid)
        dedup.append(r)
    if not dedup:
        return None

    target_name = candidate.get('full_name') or (candidate.get('name_variants') or [None])[0]
    target_country = (candidate.get('country') or '').lower()

    scored = []
    for r in dedup:
        name = r.get('display_name') or ''
        city = (r.get('city') or '').lower()
        country = (r.get('country') or '').lower()
        s = 0.70 * _similarity(target_name, name)
        if target_city and target_city == city:
            s += 0.15
        if target_country and target_country == country:
            s += 0.15
        scored.append((s, r))
    scored.sort(key=lambda x: x[0], reverse=True)

    top = [r for _, r in scored[:8]]
    chosen_id = top[0].get('id')

    if len(top) >= 2:
        s0, s1 = scored[0][0], scored[1][0]
        if s0 < 0.85 and abs(s0 - s1) < 0.06:
            llm_id = await _choose_id_with_llm(llm, candidate, 'Facebook', top)
            if isinstance(llm_id, int):
                chosen_id = llm_id

    if chosen_id is None:
        return None

    candidate_ids = [r.get('id') for r in top if r.get('id') is not None][:5]
    prof_tasks = [
        _ainvoke_with_retry(get_profile, {'user_id': int(uid)}, default=None, retries=2, timeout_s=60)
        for uid in candidate_ids
    ]
    profiles = await asyncio.gather(*prof_tasks, return_exceptions=True)

    best_id = None
    best_profile = None
    best_match = -1.0
    for uid, prof in zip(candidate_ids, profiles):
        if isinstance(prof, Exception) or not isinstance(prof, dict):
            continue
        ms = _consistency_facebook(candidate, prof)
        if ms > best_match:
            best_id = uid
            best_profile = prof
            best_match = ms

    if best_id is None or best_profile is None:
        return None

    if best_match < 0.55:
        return None
    return {
        'user_id': int(best_id),
        'profile': best_profile,
        'match_score': float(best_match),
    }

async def _score_one_cv(tools, llm, cv_text):
    candidate = await _extract_candidate(llm, cv_text)

    linkedin, facebook = await asyncio.gather(
        _best_linkedin(tools, llm, candidate),
        _best_facebook(tools, llm, candidate),
        return_exceptions=True,
    )
    if isinstance(linkedin, Exception):
        linkedin = None
    if isinstance(facebook, Exception):
        facebook = None

    linkedin_profile = None if not linkedin else linkedin.get('profile')
    facebook_profile = None if not facebook else facebook.get('profile')

    rule_score = 0.45
    if isinstance(linkedin_profile, dict):
        rule_score = _consistency_linkedin(candidate, linkedin_profile)
    elif isinstance(facebook_profile, dict):
        rule_score = _consistency_facebook(candidate, facebook_profile)

    fb_match = 0.0 if not facebook else float(facebook.get('match_score', 0.0) or 0.0)

    evidence = {
        'cv_extracted': candidate,
        'linkedin': None if not linkedin else {
            'person_id': linkedin.get('person_id'),
            'profile': _compact_linkedin(linkedin.get('profile')),
            'interactions': _compact_interactions(linkedin.get('interactions')),
            'match_score': linkedin.get('match_score'),
        },
        'facebook': None if (not facebook or fb_match < 0.60) else {
            'user_id': facebook.get('user_id'),
            'profile': _compact_facebook(facebook.get('profile')),
            'match_score': facebook.get('match_score'),
        },
    }

    prompt = f'''
You are verifying whether a CV is consistent with public social profiles.

Given the evidence JSON below, judge if the CV is reliable.

Return ONLY JSON with keys:
- score: float from 0 to 1
- verdict: one of reliable, problematic, uncertain
- key_findings: a short list of strings (max 6)

Guidelines:
- score > 0.5 means likely reliable
- Penalize clear contradictions in education, employers, titles, timelines, or location
- Do not heavily penalize missing data; treat missing as uncertainty
- If no match is found, verdict is often uncertain and score should be around 0.45 unless there are obvious red flags
- Facebook matches can be noisy; if facebook.match_score < 0.6, treat it as a likely wrong match and do not penalize the CV heavily

Evidence:
{json.dumps(evidence, ensure_ascii=False)}
'''.strip()

    resp = llm.invoke([HumanMessage(content=prompt)])
    content = getattr(resp, 'content', '') if resp is not None else ''

    score = 0.5
    verdict = 'uncertain'
    key_findings = []

    try:
        out = _extract_json(content)
        score = _clamp01(out.get('score', 0.5))
        verdict = out.get('verdict', verdict)
        key_findings = out.get('key_findings', key_findings)
        if not isinstance(key_findings, list):
            key_findings = []
    except Exception:
        score = 0.5

    score = _clamp01(0.6 * rule_score + 0.4 * score, default=rule_score)

    details = {
        'candidate': candidate,
        'linkedin_person_id': None if not linkedin else linkedin.get('person_id'),
        'facebook_user_id': None if not facebook else facebook.get('user_id'),
        'verdict': verdict,
        'key_findings': key_findings[:6],
        'rule_score': rule_score,
    }
    return score, details

async def generate_scores():
    tools_local = await _ensure_tools()
    llm_local = _ensure_llm()

    cv_paths = _find_cv_paths()
    cv_texts = [_read_pdf_text(p) for p in cv_paths]

    scores_local = []
    global verification_details
    verification_details = []

    for path, text in zip(cv_paths, cv_texts):
        try:
            s, detail = await _score_one_cv(tools_local, llm_local, text)
        except Exception as e:
            s = 0.5
            detail = {
                'candidate': {},
                'linkedin_person_id': None,
                'facebook_user_id': None,
                'verdict': 'uncertain',
                'key_findings': [f'exception: {e}'],
            }
        scores_local.append(_clamp01(s))
        detail['file'] = path
        detail['score'] = _clamp01(s)
        verification_details.append(detail)

    return scores_local

scores = await generate_scores()
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


{'decisions': [1, 1, 1, 0, 0], 'correct': 5, 'total': 5, 'final_score': 1.0}
